# 신경망
* torch.nn을 이용하여 신경망 생성해본다.
* 숫자 이미지를 분류하는 신경망을 예제로 확인


## 1. 신경망을 정의
* nn은 모델을 정의하고 미분하는데 autograd를 내부적으로 사용한다.
* nn.module은 layer와 output 반환하는 forward(input) 메서드를 포함한다.

일반적인 학습과정의 순서:
* 매개변수(weight, bias)를 갖는 신경망 정의
* dataset 입력의 반복
* forward process
* loss 계산
* graident의 back probagation
* weight 업데이트

In [121]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
#         super(Net, self).__init__()  # 이게 정석
        super().__init__()
    
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)  # 위와 같음
        x = torch.flatten(x, 1) # 1차원 기준 벡터로 flatten !!! # (1, 400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## 2 매개변수(weight, bias)의 반환이 가능하다.
매개 변수를 출력해보자.
* `list(model.parameters())`로 리스트로 받거나
* `model.named_parameters()`로 딕셔너리로 받을 수 있다.

In [15]:
params = list(net.parameters()) # generator기 때문에 list화로 리스트로 반환 가능하다.
print(len(params))
print(params[0].shape) # conv1의 .weight

10
torch.Size([6, 1, 5, 5])


In [16]:
for name, parameter in net.named_parameters():
    print(name, parameter.dtype, parameter.shape)

conv1.weight torch.float32 torch.Size([6, 1, 5, 5])
conv1.bias torch.float32 torch.Size([6])
conv2.weight torch.float32 torch.Size([16, 6, 5, 5])
conv2.bias torch.float32 torch.Size([16])
fc1.weight torch.float32 torch.Size([120, 400])
fc1.bias torch.float32 torch.Size([120])
fc2.weight torch.float32 torch.Size([84, 120])
fc2.bias torch.float32 torch.Size([84])
fc3.weight torch.float32 torch.Size([10, 84])
fc3.bias torch.float32 torch.Size([10])


## 3. 입력값을 넣어 forward, backward를 진행

In [17]:
# 배치를 고려해야 한다.
input = torch.randn(1, 1, 32, 32)
out = net(input)
out.shape

torch.Size([1, 10])

In [18]:
# backward
net.zero_grad()
# output이 scalar가 아니므로 값의 shape의 벡터를 전달
out.backward(torch.randn(1, 10))

!주의:
* `torch.nn`은 미니배치만 지원! 즉, 하나의 샘플 아닌 미니 배치를 입력으로 받는다. 따라서 `conv2d`는 (B x C x H x W)를 입력받는다.
* 만약 하나의 샘플만 있다면 `tensor.unsqueeze(0`로 한 차원 추가 가능하다.

## 4. loss function
* `nn.MSELoss`로 계산해보자.
* torch.view(shape) : np의 reshape과 같다.

In [80]:
output = net(input)
target = torch.randn(10) # 임의의 정답값이라 가정
target = target.view(1, -1)  # 한차원 늘림. (unsquieeze)와도 동일 하다.
criterion = nn.MSELoss()
loss = criterion(output, target)
loss

tensor(0.7115, grad_fn=<MseLossBackward0>)

* `.grad_fn` 속성으로 loss를 역방향 추적하면 이러한 모습의 연산그래프
```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

In [34]:
loss.grad_fn  # MSE

In [35]:
loss.grad_fn.next_functions[0][0] # Linear

In [36]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]  # relu

## 5. backprob, 역전파
* loss.backward()시, 전체 DAG가 미분되며, 그래프내의 requires_grad=True 인모든 텐서들은 gradient가 누적된 `.grad`를 갖게된다.
* `loss.backward()로 역전파 전과 후의 conv1의 biase 변수의 변화도 확인`

In [83]:
net.zero_grad() # 모든 매개변수 gradient를 0으로 초기화
print('역전파 전의 conv1 grad')
print(net.conv1.bias.grad)

loss.backward()
print('역전파 후의 conv1 grad')
print(net.conv1.bias.grad)

역전파 전의 conv1 grad
tensor([0., 0., 0., 0., 0., 0.])
역전파 후의 conv1 grad
tensor([-0.0403,  0.0148,  0.0027, -0.0133,  0.0080, -0.0215])


## 6. 가중치 갱신
optimizer로 진행. SGD, Nesterov-SGD, Adam, RMSProp 등이 `torch.optim`에 정의 되어있음

In [136]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# 아래는  traning loop에 구현해야 함.
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

--------------
### 실험.
grad는 덮어쓰기가 아니라 축적 연산이다?!

```python
grad = grad # (X)
grad += grad # (O)
```

In [134]:
# torch copy 방법
input2 = torch.ones_like(input).copy_(input)
target2 = torch.ones_like(target).copy_(target)

input2, target2

(tensor([[[[-0.1727, -0.3400, -1.0943,  ...,  1.4351, -2.2209, -1.9274],
           [ 0.5412, -0.9375, -0.6681,  ...,  0.1110,  0.2013, -0.2840],
           [-0.1223, -1.8497, -1.0430,  ...,  0.4606,  1.6043, -1.6630],
           ...,
           [ 0.0416, -0.9585, -0.1484,  ...,  0.4781,  1.9137, -1.1823],
           [-0.5759,  0.0422, -0.0964,  ...,  0.1008, -0.0169,  0.4571],
           [ 0.5812,  0.1864,  1.2260,  ...,  0.1657, -1.1614,  0.6674]]]]),
 tensor([[-0.5966,  0.1820, -0.8567,  1.1006, -1.0712,  0.1227, -0.5663,  0.3731,
          -0.8920, -1.5091]]))

실험:
* net은 zero_grad 없이 두번연속 backwward.
* net_copy는 zero_grad를 넣어 초기화함.

결과:
* net의 최종 grad는 net_copy grad의 두배. 즉, 누적됨을 확인

In [135]:
import copy
net = Net()
net_copy = copy.deepcopy(net)
output = net(input2)
loss = criterion(output, target2)
loss.backward()

output = net_copy(input2)
loss = criterion(output, target2)
loss.backward()

print('첫번째 역전파 후의 conv1 grad')
print(net.conv1.bias.grad)
print(net_copy.conv1.bias.grad)

output = net(input2)
loss = criterion(output, target2)
loss.backward()

net_copy.zero_grad()
output = net_copy(input2)
loss = criterion(output, target2)
loss.backward()
print('두번째 역전파 후의 conv1 grad')
print(net.conv1.bias.grad)
print(net_copy.conv1.bias.grad)


첫번째 역전파 후의 conv1 grad
tensor([-0.0017, -0.0135,  0.0020, -0.0056, -0.0104,  0.0008])
tensor([-0.0017, -0.0135,  0.0020, -0.0056, -0.0104,  0.0008])
두번째 역전파 후의 conv1 grad
tensor([-0.0035, -0.0269,  0.0041, -0.0112, -0.0208,  0.0016])
tensor([-0.0017, -0.0135,  0.0020, -0.0056, -0.0104,  0.0008])


In [132]:
net_copy.conv1.bias.grad * 2 

tensor([ 0.0182, -0.0053, -0.0224, -0.0109, -0.0335,  0.0049])

# 참고 :
* NN구성하는 layer, loss 함수들 doc : https://pytorch.org/docs/stable/nn.html